In [57]:
import numpy as np
from matplotlib import pyplot as plt
import copy
from scipy.linalg import expm, sinm, cosm, null_space
from scipy.signal import convolve2d
from scipy.sparse import coo_matrix
from scipy.sparse import linalg
np.seterr(divide='ignore',invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [20]:
class experiment():

    def __init__(self):
        super().__init__()
        self.T = 0.08617
        self.xrange = 10
        self.lt = 0.005
        self.ptip = [0,0,0]
        self.psample = [0,0,0]
        self.atom = [atom()]
        self.eigenvec = np.array([[-0.1332256,0,0,-0.9910857],
                         [0, 0.9829937,0.1836392,0],      
                        [0.9910857,0, 0, -0.1332256],
                        [0, -0.1836392, 0.9829937,0 ]])
        self.eigenval = np.array([0.3270655,0.7447324,5.5422276,6.8859745])
        self.position = 1
        self.jposition = 1
        self.A = 1
        self.b = 0
        self.x0 = 0
        self.y0 = 0
        self.B = [0,0,4]
        self.matrix = '1'
        self.matrixDM = '0,0,0'
        self.heisenberg_coupling  = 0
        self.sample_entanglemen = True
        self.sef = 1
        self.paramagnetic = False
        self.paramag_S = 2.5
        self.paramag_g = 2
        self.eta = 0.3
        self.no_eval = 1000
        self.max_no_eigenstates = 50
        self.third_order_calc = True
        self.rate_calc = False
        self.entanglement = False
        self.allatomsequ = 0
        self.sw = [True, False, False, True, True, False, True, False, \
                   False, False, True, False, False]
        

In [70]:
class atom():

    def __init__(self):
        super().__init__()
        self.S = 1.5
        self.g = 2
        self.D = 2.7
        self.E = 0.5
        self.J = [-0.25]
        self.U = 0
        self.w = 20
        self.DD = None
        self.G = None
    
    def init_atom(paras):
        self.S = paras[0]
        self.g = paras[1]
        self.D = paras[2]
        self.E = paras[3]
        self.J = paras[4]
        self.U = paras[5]
        self.w = paras[6]
        self.DD = paras[7]
        self.G = paras[8]

In [68]:
def Splus(S):
    a = np.arange(1,2*S.S+1)
    b = np.arange(2*S.S,0,-1)
    e = np.sqrt(a*b)
    ee = diag(e,1)
    return ee

def Sminus(S):
    a = np.arange(1,2*S.S+1)
    b = np.arange(2*S.S,0,-1)
    e = np.sqrt(a*b)
    ee = diag(e,-1)
    return ee

def Sx(S):
    return 0.5*(Splus(S)+Sminus(S))
def Sy(S):
    return -0.5*1j*(Splus(S)-Sminus(S))
def Sz(S):
    return diag(np.arange(S.S,-S.S-1,-1))
def S1(S):
    return diag(np.ones((int(S.S*2)+1,1)).flatten().tolist())

In [121]:
def HZ(S, B):
    a = len(B)
    if a != 1 and a != 3:
        print('error5')
        return
    if a==1:
        B = [0,0,B[0]]
    e = S.g*0.05788*(B[0]*Sx(S)+B[1]*Sy(S)+B[2]*Sz(S))
    return e

In [62]:
def Haniso(S):
    if S.D == None:
        S.D = 0
    if S.E == None:
        S.E = 0
    e = S.D*np.dot(Sz(S),Sz(S))+S.E*(np.dot(Sx(S), Sx(S))-np.dot(Sy(S),Sy(S)))
    if S.DD != None:
        e = e+S.DD*np,power(Sz(S), 4.)
    if S.G != None:
        e = e+S.G*(np.power(Splus(S),4.)+np.power(Sminus(S),4.))
    return e

In [15]:
def HHeisenberg(t,J,n,m):
    a = np.sum(J.shape)
    if a != 1 and a != 3:
        print('error5')
        return
    if a==1:
        J = [J, J, J]
    b = t.shape[1]
    if (b<n or b<m) or n==m:
        print('error1')
        return
    if n>m:
        p=copy.deepcopy(n); n=copy.deepcopy(m); m=copy.deepcopy(p)
        
    x=1;y=1;z=1
    for j in range(n-1):
        m1=sparse(S1(t[j]))
        x = np.kron(x, m1)
        y = np.kron(y, m1)
        z = np.kron(z, m1)
    x = np.kron(x, Sx(t[n-1]))
    y = np.kron(y, Sy(t[n-1]))
    z = np.kron(z, Sz(t[n-1]))
    for j in range(n, m-1):
        m1 = sparse(S1(t[j]))
        x = np.kron(x, m1)
        y = np.kron(y, m1)
        z = np.kron(z, m1)
    x = np.kron(x, Sx(t[m-1]))
    y = np.kron(y, Sy(t[m-1]))
    z = np.kron(z, Sz(t[m-1]))
    for j in range(m, b):
        m1 = sparse(S1(t[j]))
        x = np.kron(x, m1)
        y = np.kron(y, m1)
        z = np.kron(z, m1)
    e = J[0]*x+J[1]*y+J[2]*z
    return e

In [17]:
def HDM(t, D, n, m):
    b = t.shape[1]
    if (b<n or b<m) or n==m:
        print('error1')
        return
    if n>m:
        p=copy.deepcopy(n); n=copy.deepcopy(m)
        m=copy.deepcopy(p); D=copy.deepcopy(-D)
    x=1;y=1;z=1;xx=1;yy=1;zz=1
    
    for j in range(n-1):
        x = np.kron(x, sparse(S1(t[j])))
        xx = copy.deepcopy(x); y = copy.deepcopy(x); yy = copy.deepcopy(x)
        z = copy.deepcopy(x); zz = copy.deepcopy(x)
    x = np.kron(x, sparse(Sy(t[n-1]))); xx = np,kron(xx, sparse(Sz(t[n-1])))
    y = np.kron(y, sparse(Sz(t[n-1]))); yy = np,kron(yy, sparse(Sx(t[n-1])))
    z = np.kron(z, sparse(Sx(t[n-1]))); zz = np,kron(zz, sparse(Sy(t[n-1])))
    for j in range(n+1,m):
        m1 = sparse(S1(t[j]))
        x = np.kron(x, m1); xx = np.kron(xx, m1)
        y = np.kron(y, m1); yy = np.kron(yy, m1)
        z = np.kron(z, m1); zz = np.kron(zz, m1)
    x = np.kron(x, sparse(Sz(t[m-1]))); xx = np,kron(xx, sparse(Sy(t[n-1])))
    y = np.kron(y, sparse(Sx(t[m-1]))); yy = np,kron(yy, sparse(Sz(t[n-1])))
    z = np.kron(z, sparse(Sy(t[m-1]))); zz = np,kron(zz, sparse(Sx(t[n-1]))) 
    for j in range(m+1,b+1):
        m1 = sparse(S1(t[j]))
        x = np.kron(x, m1); xx = np.kron(xx, m1)
        y = np.kron(y, m1); yy = np.kron(yy, m1)
        z = np.kron(z, m1); zz = np.kron(zz, m1)
    e = D[0]*(x-xx)+D[1]*(y-yy)+D[2]*(z-zz)
    return e

In [66]:
def HAni(t, B):
    b = len(t)# need to modify for multiple spins!!!
    if b == 1:
        e = Haniso(t[0])+HZ(t[0],B)
    else:
        e = []
        for i in range(b):
            x = 1
            for j in range(i-1):
                x = np.kron(x, sparse(S1(t[j])))
            x = np.kron(x, sparse(Haniso(t[i])+HZ(t[i],B)))
            for j in range(i, b):
                x = np.kron(x, sparse(S1(t[j])))
            e = e+x
    return e

In [21]:
def hamiltonian(exp):
    H = HAni(exp.atom, exp.B)
    
    if len(exp.atom) > 1:
        if exp.matrix != None:
            M = exp.matrix
            J = exp.heisenberg_coupling
            if J != 0:
                for i in range(M.shape[0]):
                    for j in range(1, M.shape[1]+1):
                        if j>i:
                            H = H+HHeisenberg(exp.atom. evstr(M[i,j-1])*J, i, j)
        if exp.matrixDM != None:
            M = exp.matrixDM
            J = exp.heisenberg_coupling
            if J != 0:
                for i in range(M.shape[0]):
                    for j in range(1,M.shape[1]+1):
                        if j>i:
                            H = H+HDM(exp.atom, evstr(M[i,j-1])*J,i,j)
    return H

In [118]:
def eigenvalues(exp):
    H = hamiltonian(exp)
    if exp.max_no_eigenstates != None:
        if H.shape[0] < exp.max_no_eigenstates+2:
            [exp.eigenvec, exp.eigenval] = spec(H)
        else:
            ofst=0; ofstold=0
            while ofst>= 0:
                H = H-ofst*speye(H)
                [exp.eigenvec, exp.eigenval] = eigs(H, speye(H), exp.max_no_eigenstates)
                ofstold = ofstold+ofst
                ofst = np.max(np.real((diag(exp.eigenval))))
                if ofstold > 0:
                    exp.eigenval = exp.eigenval+ofstold*eye(exp.eigenval)
        [exp.eigenvec, exp.eigenval] = spec(H)
    exp.eigenval = diag(np.real(exp.eigenval))
    exp.eigenvec = clean(exp.eigenvec)
    sort_eigenval()

In [119]:
def sort_eigenval():
    evals = exp.eigenval.diagonal(0)
    ind = np.unravel_index(np.argsort(evals), evals.shape)
    exp.eigenval = evals[ind]
    index = np.lexsort((evals,))
    exp.eigenvec = exp.eigenvec.T[index].T

In [54]:
def speye(*par):
    if len(par) == 1:
        [m,n] = par[0].shape
    else:
        m = par[0]; n = par[1]

        mn = np.min([m, n])
    sp = coo_matrix((np.ones(mn), (np.arange(0,mn), np.arange(0,mn))), shape=(m,n))
    return sp.toarray()

In [55]:
def sparse(*par):
    if len(par) == 1:
        S = par[0]
        return Matrix(S).tocoo().toarray()
#     coo_matrix((data, (row, col)), shape=(4, 4))

    print03.__code__.co_argcount

In [58]:
def eigs(A, M, k):
    return linalg.eigs(A, M=M, k=k)

In [22]:
def spec(m):
    d = np.linalg.eig(m)
    return d[1], d[0]

In [26]:
def clean(m, esp=1e-10):
    m[abs(m)<esp] = 0
    return m

In [27]:
def diag(evals, k=0):
    e = np.zeros((len(evals)+abs(k),len(evals)+abs(k)))
    for i in range(e.shape[0]):
        if i+k<=e.shape[0]-1 and i+k>=0:
            e[i,i+k] = evals[i+k] if k<=0 else evals[i]
        else:
            continue
    return e

In [120]:
exp = experiment()
print('---',exp.eigenval,'\n', exp.eigenvec)
eigenvalues(exp)
print('---',exp.eigenval,'\n', exp.eigenvec)

--- [0.3270655 0.7447324 5.5422276 6.8859745] 
 [[-0.1332256  0.         0.        -0.9910857]
 [ 0.         0.9829937  0.1836392  0.       ]
 [ 0.9910857  0.         0.        -0.1332256]
 [ 0.        -0.1836392  0.9829937  0.       ]]
--- [0.32706547 0.7447324  5.5422276  6.88597453] 
 [[-0.13322563+0.j  0.        +0.j  0.        +0.j  0.99108573+0.j]
 [ 0.        +0.j  0.98299372+0.j  0.18363918+0.j  0.        +0.j]
 [ 0.99108573+0.j  0.        +0.j  0.        +0.j  0.13322563-0.j]
 [ 0.        +0.j -0.18363918+0.j  0.98299372+0.j  0.        +0.j]]
